In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import os

# 설정
IMAGE_SIZE = (128, 256)
BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 1e-5
DATA_DIR = 'finetune_data_s1_simple'
MODEL_SAVE_PATH = 'stage1_finetuned_simple.h5'

# 데이터 로드
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_gen = datagen.flow_from_directory(DATA_DIR, target_size=IMAGE_SIZE, class_mode='binary',
                                        batch_size=BATCH_SIZE, subset='training')
val_gen = datagen.flow_from_directory(DATA_DIR, target_size=IMAGE_SIZE, class_mode='binary',
                                      batch_size=BATCH_SIZE, subset='validation')

# 클래스 가중치 계산
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_gen.classes), y=train_gen.classes)
class_weights = dict(enumerate(class_weights))

# 모델 구성
base_model = ResNet50V2(include_top=False, weights='imagenet', input_shape=(*IMAGE_SIZE, 3))
base_model.trainable = False  # 처음엔 동결

inputs = Input(shape=(*IMAGE_SIZE, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation='sigmoid')(x)
model = Model(inputs, outputs)

# 컴파일
model.compile(optimizer=tf.keras.optimizers.Adam(LEARNING_RATE),
              loss='binary_crossentropy', metrics=['accuracy'])

# 콜백
callbacks = [
    ModelCheckpoint(MODEL_SAVE_PATH, save_best_only=True, monitor='val_accuracy', mode='max'),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7)
]

# 학습
model.fit(train_gen, validation_data=val_gen, epochs=EPOCHS,
          class_weight=class_weights, callbacks=callbacks)

# 평가
loss, acc = model.evaluate(val_gen)
print(f"Val Accuracy: {acc:.4f}")


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import os

# 설정
IMAGE_SIZE = (128, 256)
BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 1e-5
DATA_DIR = 'finetune_data_s2_simple'
MODEL_SAVE_PATH = 'stage2_finetuned_simple.h5'

# 데이터 로드
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_gen = datagen.flow_from_directory(DATA_DIR, target_size=IMAGE_SIZE, class_mode='categorical',
                                        batch_size=BATCH_SIZE, subset='training')
val_gen = datagen.flow_from_directory(DATA_DIR, target_size=IMAGE_SIZE, class_mode='categorical',
                                      batch_size=BATCH_SIZE, subset='validation')

# 클래스 가중치 계산
y_integers = np.argmax(train_gen.labels, axis=1) if hasattr(train_gen, 'labels') else train_gen.classes
class_weights_arr = compute_class_weight(class_weight='balanced', classes=np.unique(y_integers), y=y_integers)
class_weights = dict(enumerate(class_weights_arr))

# 모델 구성
base_model = ResNet50V2(include_top=False, weights='imagenet', input_shape=(*IMAGE_SIZE, 3))
base_model.trainable = False

inputs = Input(shape=(*IMAGE_SIZE, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
outputs = Dense(3, activation='softmax')(x)
model = Model(inputs, outputs)

# 컴파일
model.compile(optimizer=tf.keras.optimizers.Adam(LEARNING_RATE),
              loss='categorical_crossentropy', metrics=['accuracy'])

# 콜백
callbacks = [
    ModelCheckpoint(MODEL_SAVE_PATH, save_best_only=True, monitor='val_accuracy', mode='max'),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7)
]

# 학습
model.fit(train_gen, validation_data=val_gen, epochs=EPOCHS,
          class_weight=class_weights, callbacks=callbacks)

# 평가
loss, acc = model.evaluate(val_gen)
print(f"Val Accuracy: {acc:.4f}")


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# 공통 설정
IMAGE_SIZE = (128, 256)
BATCH_SIZE = 32
EPOCHS_INITIAL = 5
EPOCHS_FINETUNE = 5
LR_INITIAL = 1e-5
LR_FINETUNE = 1e-6

def build_model(num_classes, input_shape=(128, 256, 3)):
    base_model = ResNet50V2(include_top=False, weights='imagenet', input_shape=input_shape)
    base_model.trainable = False
    inputs = Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation='softmax' if num_classes > 1 else 'sigmoid')(x)
    model = Model(inputs, outputs)
    return model, base_model

def plot_confusion_matrix(cm, class_names, title):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(title)
    plt.tight_layout()
    plt.savefig(f'{title.replace(" ", "_")}.png')
    plt.close()

def train_stage(stage_name, data_dir, num_classes, is_binary):
    model_path = f'{stage_name}_finetuned_model.h5'

    datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
    mode = 'binary' if is_binary else 'categorical'

    train_gen = datagen.flow_from_directory(data_dir, target_size=IMAGE_SIZE,
                                            class_mode=mode, batch_size=BATCH_SIZE,
                                            subset='training', shuffle=True)
    val_gen = datagen.flow_from_directory(data_dir, target_size=IMAGE_SIZE,
                                          class_mode=mode, batch_size=BATCH_SIZE,
                                          subset='validation', shuffle=False)

    y_labels = train_gen.classes if is_binary else np.argmax(train_gen.labels, axis=1)
    weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_labels), y=y_labels)
    class_weights = dict(enumerate(weights))

    model, base_model = build_model(num_classes)
    loss = 'binary_crossentropy' if is_binary else 'categorical_crossentropy'
    model.compile(optimizer=tf.keras.optimizers.Adam(LR_INITIAL), loss=loss, metrics=['accuracy'])

    callbacks = [
        ModelCheckpoint(model_path, monitor='val_accuracy', save_best_only=True, mode='max'),
        EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7)
    ]

    print(f"🔧 [{stage_name}] Initial training...")
    model.fit(train_gen, validation_data=val_gen, epochs=EPOCHS_INITIAL, callbacks=callbacks,
              class_weight=class_weights)

    print(f"🔁 [{stage_name}] Unfreezing base model and fine-tuning...")
    base_model.trainable = True
    for layer in base_model.layers:
        if isinstance(layer, tf.keras.layers.BatchNormalization):
            layer.trainable = False

    model.compile(optimizer=tf.keras.optimizers.Adam(LR_FINETUNE), loss=loss, metrics=['accuracy'])
    model.fit(train_gen, validation_data=val_gen, epochs=EPOCHS_FINETUNE, callbacks=callbacks,
              class_weight=class_weights)

    print(f"📈 [{stage_name}] Evaluating...")
    model.load_weights(model_path)
    val_gen.reset()
    preds = model.predict(val_gen)
    y_true = val_gen.classes if is_binary else np.argmax(val_gen.labels, axis=1)
    y_pred = (preds > 0.5).astype(int).ravel() if is_binary else np.argmax(preds, axis=1)

    class_names = list(val_gen.class_indices.keys())
    print(f"\n📄 [{stage_name}] Classification Report:")
    print(classification_report(y_true, y_pred, target_names=class_names))

    cm = confusion_matrix(y_true, y_pred)
    plot_confusion_matrix(cm, class_names, f'{stage_name} Confusion Matrix')
    print(f"✅ [{stage_name}] 완료 및 시각화 저장됨.\n")

if __name__ == '__main__':
    stages = [
        ('Stage1', 'finetune_data_s1_simple', 1, True),
        ('Stage2', 'finetune_data_s2_simple', 3, False),
    ]
    processes = []
    for args in stages:
        p = multiprocessing.Process(target=train_stage, args=args)
        p.start()
        processes.append(p)
    for p in processes:
        p.join()


## 이거 1개 실행하면 됨

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 설정
IMAGE_SIZE = (128, 256)
BATCH_SIZE = 32
VAL_ACC_TARGET = 0.89
MODEL_PATHS = {
    'Stage1': 'anomaly_model/stage1_tl_best_model_resnet.h5',
    'Stage2': 'anomaly_model/stage2_tl_best_model_resnet.h5',
}
DATA_PATHS = {
    'Stage1': 'finetune_data_s1_simple',
    'Stage2': 'finetune_data_s2_simple',
}
IS_BINARY = {
    'Stage1': True,
    'Stage2': False,
}
NUM_CLASSES = {
    'Stage1': 1,
    'Stage2': 3,
}

class StopAtValAcc(tf.keras.callbacks.Callback):
    def __init__(self, target=0.89):
        super().__init__()
        self.target = target

    def on_epoch_end(self, epoch, logs=None):
        val_acc = logs.get("val_accuracy")
        if val_acc and val_acc >= self.target:
            print(f"\n✅ 목표 val_accuracy {self.target} 도달. 학습 조기 종료.")
            self.model.stop_training = True

def plot_confusion_matrix(cm, class_names, title):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(title)
    plt.tight_layout()
    plt.savefig(f'{title.replace(" ", "_")}.png')
    plt.close()

def finetune_stage(stage):
    print(f"\n🚀 [START] {stage} fine-tuning 시작")

    model = load_model(MODEL_PATHS[stage])
    data_path = DATA_PATHS[stage]
    is_binary = IS_BINARY[stage]
    num_classes = NUM_CLASSES[stage]

    datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
    mode = 'binary' if is_binary else 'categorical'

    train_gen = datagen.flow_from_directory(
        data_path, target_size=IMAGE_SIZE, class_mode=mode,
        color_mode='grayscale',  # ✅ 변경
        batch_size=BATCH_SIZE, subset='training', shuffle=True
    )
    val_gen = datagen.flow_from_directory(
        data_path, target_size=IMAGE_SIZE, class_mode=mode,
        color_mode='grayscale',  # ✅ 변경
        batch_size=BATCH_SIZE, subset='validation', shuffle=False
    )

    y_labels = train_gen.classes
    weights = compute_class_weight(class_weight='balanced',
                                   classes=np.unique(y_labels), y=y_labels)
    class_weights = dict(enumerate(weights))

    loss_fn = 'binary_crossentropy' if is_binary else 'categorical_crossentropy'
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
                  loss=loss_fn,
                  metrics=['accuracy'])

    callbacks = [
        StopAtValAcc(target=VAL_ACC_TARGET),
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)
    ]

    model.fit(train_gen, validation_data=val_gen, epochs=30,
              callbacks=callbacks, class_weight=class_weights)

    print(f"\n📈 [EVAL] {stage} 모델 평가 시작")
    val_gen.reset()
    preds = model.predict(val_gen)
    y_true = val_gen.classes
    y_pred = (preds > 0.5).astype(int).ravel() if is_binary else np.argmax(preds, axis=1)

    class_names = list(val_gen.class_indices.keys())
    print(f"\n📄 [{stage}] Classification Report:")
    print(classification_report(y_true, y_pred, target_names=class_names))

    # F1-score 직접 출력
    f1_macro = f1_score(y_true, y_pred, average='macro')
    f1_weighted = f1_score(y_true, y_pred, average='weighted')
    print(f"🧮 [{stage}] F1-score (macro): {f1_macro:.4f}")
    print(f"🧮 [{stage}] F1-score (weighted): {f1_weighted:.4f}")

    cm = confusion_matrix(y_true, y_pred)
    plot_confusion_matrix(cm, class_names, f'{stage} Confusion Matrix')
    print(f"✅ [{stage}] 완료 및 시각화 저장됨.")

if __name__ == '__main__':
    finetune_stage('Stage1')
    finetune_stage('Stage2')



🚀 [START] Stage1 fine-tuning 시작


Found 160 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


c:\Users\jh\.conda\envs\tf-gpu\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.0000e+00 - loss: 8.5217 - val_accuracy: 0.0000e+00 - val_loss: 7.6898 - learning_rate: 1.0000e-05
Epoch 2/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.0098 - loss: 7.5144 - val_accuracy: 0.0000e+00 - val_loss: 7.0984 - learning_rate: 1.0000e-05
Epoch 3/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.0355 - loss: 7.0886 - val_accuracy: 0.0000e+00 - val_loss: 6.5244 - learning_rate: 1.0000e-05
Epoch 4/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.0312 - loss: 6.0607 - val_accuracy: 0.0000e+00 - val_loss: 5.9553 - learning_rate: 1.0000e-05
Epoch 5/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.0758 - loss: 6.0845 - val_accuracy: 0.0000e+00 - val_loss: 5.3803 - learning_rate: 1.0000e-05
Epoch 6/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.1464 - loss: 5.8874 - val_accuracy: 0.0250 - val_loss: 4.7859 - learning_rate: 1.0000e-05
Epoch 7/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.2102 - l

Found 360 images belonging to 3 classes.
Found 90 images belonging to 3 classes.


c:\Users\jh\.conda\envs\tf-gpu\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step - accuracy: 0.1822 - loss: 16.7756 - val_accuracy: 0.3111 - val_loss: 12.9268 - learning_rate: 1.0000e-05
Epoch 2/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.2489 - loss: 14.4211 - val_accuracy: 0.3444 - val_loss: 11.1092 - learning_rate: 1.0000e-05
Epoch 3/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.3205 - loss: 12.2038 - val_accuracy: 0.3556 - val_loss: 9.3532 - learning_rate: 1.0000e-05
Epoch 4/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.2919 - loss: 11.9695 - val_accuracy: 0.4556 - val_loss: 7.3968 - learning_rate: 1.0000e-05
Epoch 5/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.3760 - loss: 8.9587 - val_accuracy: 0.5889 - val_loss: 5.4217 - learning_rate: 1.0000e-05
Epoch 6/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - accuracy: 0.4133 - loss: 6.6414 - val_accuracy: 0.6556 - val_loss: 3.5284 - learning_rate: 1.0000e-05
Epoch 7/30
12/12 ━━━━━━━━━━━━━━━━━━━━ 12s 999ms/step - accuracy: 0.464

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 설정
IMAGE_SIZE = (128, 256)
BATCH_SIZE = 32
VAL_ACC_TARGET = 0.89
MODEL_PATHS = {
    'Stage1': 'anomaly_model/stage1_tl_best_model_resnet.h5',
    'Stage2': 'anomaly_model/stage2_tl_best_model_resnet.h5',
}
DATA_PATHS = {
    'Stage1': 'finetune_data_split_s1',
    'Stage2': 'finetune_data_split_s2',
}
IS_BINARY = {
    'Stage1': True,
    'Stage2': False,
}
NUM_CLASSES = {
    'Stage1': 1,
    'Stage2': 3,
}

class StopAtValAcc(tf.keras.callbacks.Callback):
    def __init__(self, target=0.89):
        super().__init__()
        self.target = target

    def on_epoch_end(self, epoch, logs=None):
        val_acc = logs.get("val_accuracy")
        if val_acc and val_acc >= self.target:
            print(f"\n✅ 목표 val_accuracy {self.target} 도달. 학습 조기 종료.")
            self.model.stop_training = True

def plot_confusion_matrix(cm, class_names, title):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title(title)
    plt.tight_layout()
    plt.savefig(f'{title.replace(" ", "_")}.png')
    plt.close()

def finetune_stage(stage):
    print(f"\n🚀 [START] {stage} fine-tuning 시작")

    model = load_model(MODEL_PATHS[stage])
    data_path = DATA_PATHS[stage]
    is_binary = IS_BINARY[stage]
    num_classes = NUM_CLASSES[stage]

    datagen = ImageDataGenerator(rescale=1./255)
    mode = 'binary' if is_binary else 'categorical'

    train_gen = datagen.flow_from_directory(
        os.path.join(data_path, 'train'), target_size=IMAGE_SIZE, class_mode=mode,
        color_mode='grayscale', batch_size=BATCH_SIZE, shuffle=True
    )
    val_gen = datagen.flow_from_directory(
        os.path.join(data_path, 'val'), target_size=IMAGE_SIZE, class_mode=mode,
        color_mode='grayscale', batch_size=BATCH_SIZE, shuffle=False
    )
    test_gen = datagen.flow_from_directory(
        os.path.join(data_path, 'test'), target_size=IMAGE_SIZE, class_mode=mode,
        color_mode='grayscale', batch_size=BATCH_SIZE, shuffle=False
    )

    y_labels = train_gen.classes
    weights = compute_class_weight(class_weight='balanced',
                                   classes=np.unique(y_labels), y=y_labels)
    class_weights = dict(enumerate(weights))

    loss_fn = 'binary_crossentropy' if is_binary else 'categorical_crossentropy'
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
                  loss=loss_fn,
                  metrics=['accuracy'])

    callbacks = [
        StopAtValAcc(target=VAL_ACC_TARGET),
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)
    ]

    model.fit(train_gen, validation_data=val_gen, epochs=30,
              callbacks=callbacks, class_weight=class_weights)
    
        # 모델 저장
    save_path = f'anomaly_model/{stage.lower()}_finetuned_model.h5'
    model.save(save_path)
    print(f"💾 [{stage}] Fine-tuned 모델 저장 완료 → {save_path}")


    print(f"\n📈 [EVAL] {stage} 모델 평가 시작")
    test_gen.reset()
    preds = model.predict(test_gen)
    y_true = test_gen.classes
    y_pred = (preds > 0.5).astype(int).ravel() if is_binary else np.argmax(preds, axis=1)

    class_names = list(test_gen.class_indices.keys())
    print(f"\n📄 [{stage}] Classification Report:")
    print(classification_report(y_true, y_pred, target_names=class_names))

    f1_macro = f1_score(y_true, y_pred, average='macro')
    f1_weighted = f1_score(y_true, y_pred, average='weighted')
    print(f"🧮 [{stage}] F1-score (macro): {f1_macro:.4f}")
    print(f"🧮 [{stage}] F1-score (weighted): {f1_weighted:.4f}")

    cm = confusion_matrix(y_true, y_pred)
    plot_confusion_matrix(cm, class_names, f'{stage} Confusion Matrix')
    print(f"✅ [{stage}] 완료 및 시각화 저장됨.")

if __name__ == '__main__':
    finetune_stage('Stage1')
    finetune_stage('Stage2')



🚀 [START] Stage1 fine-tuning 시작


Found 180 images belonging to 2 classes.
Found 60 images belonging to 2 classes.
Found 60 images belonging to 2 classes.


c:\Users\jh\.conda\envs\tf-gpu\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.0000e+00 - loss: 8.2368 - val_accuracy: 0.0000e+00 - val_loss: 9.5991 - learning_rate: 1.0000e-05
Epoch 2/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.0109 - loss: 7.5038 - val_accuracy: 0.0000e+00 - val_loss: 8.8042 - learning_rate: 1.0000e-05
Epoch 3/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.0136 - loss: 7.0081 - val_accuracy: 0.0000e+00 - val_loss: 7.9981 - learning_rate: 1.0000e-05
Epoch 4/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.0458 - loss: 6.1126 - val_accuracy: 0.0000e+00 - val_loss: 7.1717 - learning_rate: 1.0000e-05
Epoch 5/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.1244 - loss: 5.5001 - val_accuracy: 0.0833 - val_loss: 6.2929 - learning_rate: 1.0000e-05
Epoch 6/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.1425 - loss: 4.1306 - val_accuracy: 0.1667 - val_loss: 5.3695 - learning_rate: 1.0000e-05
Epoch 7/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - accuracy: 0.1785 - loss:

Found 450 images belonging to 3 classes.
Found 150 images belonging to 3 classes.
Found 150 images belonging to 3 classes.


c:\Users\jh\.conda\envs\tf-gpu\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.2141 - loss: 17.4315 - val_accuracy: 0.2933 - val_loss: 12.8080 - learning_rate: 1.0000e-05
Epoch 2/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.2279 - loss: 13.9677 - val_accuracy: 0.3467 - val_loss: 10.5359 - learning_rate: 1.0000e-05
Epoch 3/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.3100 - loss: 12.1353 - val_accuracy: 0.4600 - val_loss: 8.2995 - learning_rate: 1.0000e-05
Epoch 4/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.3629 - loss: 9.4852 - val_accuracy: 0.5533 - val_loss: 5.9338 - learning_rate: 1.0000e-05
Epoch 5/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.4085 - loss: 7.6111 - val_accuracy: 0.6200 - val_loss: 3.6343 - learning_rate: 1.0000e-05
Epoch 6/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 17s 1s/step - accuracy: 0.4793 - loss: 5.5685 - val_accuracy: 0.6867 - val_loss: 1.7750 - learning_rate: 1.0000e-05
Epoch 7/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.5877 - 

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Reshape, Dropout, Dense, LSTM
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# ------------------------- 설정 -------------------------
IMG_SIZE = (128, 256)
BATCH_SIZE = 32
EPOCHS = 50
LR = 1e-4

# 데이터 경로 설정
data_root = "/Users/pjh_air/Documents/SJ_simhwa/final/1/Split_Spectrograms_Data"
train_stage1_path = os.path.join(data_root, "train/stage1")
test_stage1_path = os.path.join(data_root, "test/stage1")
train_stage2_path = os.path.join(data_root, "train/stage2")
test_stage2_path = os.path.join(data_root, "test/stage2")

# ------------------------- 도메인 레이블 생성 함수 -------------------------
def domain_label_generator(generator, domain_label):
    """Label Generator를 기반으로 도메인 라벨 생성 (0 또는 1)"""
    while True:
        x, y = next(generator)
        domain_labels = tf.keras.utils.to_categorical(
            [domain_label] * x.shape[0], num_classes=2
        )
        yield x, {'label_output': y, 'domain_output': domain_labels}

# ------------------------- MCDANN 모델 정의 -------------------------
class GradientReversal(tf.keras.layers.Layer):
    def __init__(self, lambda_=1.0, **kwargs):
        super().__init__(**kwargs)
        self.lambda_ = lambda_

    def call(self, x):
        @tf.custom_gradient
        def reverse_gradient(x):
            def grad(dy):
                return -self.lambda_ * dy
            return x, grad
        return reverse_gradient(x)

def build_mcdann_model(input_shape, num_classes, is_binary=False):
    input_tensor = Input(shape=input_shape)
    x = Conv2D(3, (1,1), padding='same', name='gray_to_rgb')(input_tensor)

    base_model = ResNet50V2(include_top=False, weights='imagenet')
    x = base_model(x)  # (None, 4, 8, 2048)
    x = Reshape((8, 4 * 2048))(x)  # (None, 8, 8192)
    x = LSTM(128)(x)
    x = Dropout(0.5)(x)

    # Label head
    if is_binary:
        label_output = Dense(1, activation='sigmoid', name='label_output')(x)
    else:
        label_output = Dense(num_classes, activation='softmax', name='label_output')(x)

    # Domain head
    grl = GradientReversal()(x)
    domain_output = Dense(2, activation='softmax', name='domain_output')(grl)

    return Model(inputs=input_tensor, outputs=[label_output, domain_output])

# ------------------------- 평가 함수 정의 -------------------------
def evaluate_label_only(model, generator, title, is_binary=False):
    y_pred_probs, _ = model.predict(generator)
    y_true = generator.classes
    if is_binary:
        y_pred = (y_pred_probs > 0.5).astype(int)
    else:
        y_pred = np.argmax(y_pred_probs, axis=1)

    class_names = list(generator.class_indices.keys())
    print(f"\n--- {title} Classification Report ---")
    print(classification_report(y_true, y_pred, target_names=class_names))

    print(f"\n--- {title} Confusion Matrix ---")
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title(f"{title} Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.show()

# ------------------------- 데이터 로딩 -------------------------
datagen = ImageDataGenerator(rescale=1./255)

train_gen_s1 = datagen.flow_from_directory(train_stage1_path, target_size=IMG_SIZE, color_mode='grayscale',
                                           batch_size=BATCH_SIZE, class_mode='binary')
test_gen_s1 = datagen.flow_from_directory(test_stage1_path, target_size=IMG_SIZE, color_mode='grayscale',
                                          batch_size=BATCH_SIZE, class_mode='binary', shuffle=False)

train_gen_s2 = datagen.flow_from_directory(train_stage2_path, target_size=IMG_SIZE, color_mode='grayscale',
                                           batch_size=BATCH_SIZE, class_mode='categorical')
test_gen_s2 = datagen.flow_from_directory(test_stage2_path, target_size=IMG_SIZE, color_mode='grayscale',
                                          batch_size=BATCH_SIZE, class_mode='categorical', shuffle=False)

# ------------------------- Stage 1 학습 -------------------------
print("\n[Stage 1 MCDANN 학습 시작]")
model_s1 = build_mcdann_model((128, 256, 1), num_classes=1, is_binary=True)
model_s1.compile(
    optimizer=Adam(LR),
    loss={'label_output': 'binary_crossentropy', 'domain_output': 'categorical_crossentropy'},
    loss_weights={'label_output': 1.0, 'domain_output': 0.1},
    metrics={'label_output': 'accuracy', 'domain_output': 'accuracy'}
)

model_s1.fit(domain_label_generator(train_gen_s1, domain_label=0),
             steps_per_epoch=len(train_gen_s1),
             epochs=EPOCHS,
             callbacks=[EarlyStopping(patience=5, restore_best_weights=True)])

evaluate_label_only(model_s1, test_gen_s1, title="Stage 1 MCDANN", is_binary=True)

# ------------------------- Stage 2 학습 -------------------------
print("\n[Stage 2 MCDANN 학습 시작]")
model_s2 = build_mcdann_model((128, 256, 1), num_classes=train_gen_s2.num_classes, is_binary=False)
model_s2.compile(
    optimizer=Adam(LR),
    loss={'label_output': 'categorical_crossentropy', 'domain_output': 'categorical_crossentropy'},
    loss_weights={'label_output': 1.0, 'domain_output': 0.1},
    metrics={'label_output': 'accuracy', 'domain_output': 'accuracy'}
)

model_s2.fit(domain_label_generator(train_gen_s2, domain_label=0),
             steps_per_epoch=len(train_gen_s2),
             epochs=EPOCHS,
             callbacks=[EarlyStopping(patience=5, restore_best_weights=True)])

evaluate_label_only(model_s2, test_gen_s2, title="Stage 2 MCDANN", is_binary=False)

print("\n✅ 모든 MCDANN 모델 학습 및 평가 완료")


val curve 시각화 (acc/loss)

예측결과 CSV 저장

threshold 튜닝 포함

In [ ]:

import os
import numpy as np
import tensorflow as tf
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import (Input, Reshape, LSTM, Dropout, Dense)
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50V2
import matplotlib.pyplot as plt
import seaborn as sns

# Gradient Reversal Layer
@tf.custom_gradient
def grad_reverse(x):
    def custom_grad(dy):
        return -dy
    return x, custom_grad

class GradientReversal(tf.keras.layers.Layer):
    def call(self, x):
        return grad_reverse(x)

# 설정
IMAGE_SIZE = (128, 256)
USE_FINE_TUNING = True  # False면 ResNet 동결
BATCH_SIZE = 32
EPOCHS = 30
DATA_PATH = 'finetune_data_split_s2'
NUM_CLASSES = 3
REPORT_DIR = 'reports'
os.makedirs(REPORT_DIR, exist_ok=True)

# 데이터
datagen = ImageDataGenerator(rescale=1./255)
train_gen = datagen.flow_from_directory(
    os.path.join(DATA_PATH, 'train'), target_size=IMAGE_SIZE, class_mode='categorical',
    color_mode='rgb', batch_size=BATCH_SIZE, shuffle=True)
val_gen = datagen.flow_from_directory(
    os.path.join(DATA_PATH, 'val'), target_size=IMAGE_SIZE, class_mode='categorical',
    color_mode='rgb', batch_size=BATCH_SIZE, shuffle=False)
test_gen = datagen.flow_from_directory(
    os.path.join(DATA_PATH, 'test'), target_size=IMAGE_SIZE, class_mode='categorical',
    color_mode='rgb', batch_size=BATCH_SIZE, shuffle=False)

# 입력 정의
inputs = Input(shape=(128, 256, 3))
base_model = ResNet50V2(include_top=False, weights='imagenet', input_tensor=inputs)

# Fine-tuning 제어
if not USE_FINE_TUNING:
    base_model.trainable = False
else:
    for layer in base_model.layers:
        if not isinstance(layer, tf.keras.layers.BatchNormalization):
            layer.trainable = True

x = base_model.output  # shape: (None, h, w, c)
x = Reshape((x.shape[1], x.shape[2] * x.shape[3]))(x)  # (None, time, features)
x = LSTM(128)(x)
x = Dropout(0.5)(x)

# Label Head
label_output = Dense(NUM_CLASSES, activation='softmax', name='label_head')(x)

# Domain Head with GRL
grl = GradientReversal()(x)
domain_output = Dense(100, activation='relu')(grl)
domain_output = Dense(2, activation='softmax', name='domain_head')(domain_output)

# 모델 구성
model = Model(inputs, [label_output, domain_output])
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss={'label_head': 'categorical_crossentropy', 'domain_head': 'categorical_crossentropy'},
              metrics={'label_head': 'accuracy', 'domain_head': 'accuracy'})

# 클래스 가중치
y_labels = train_gen.classes
weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_labels), y=y_labels)
class_weights = dict(enumerate(weights))

# 콜백
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_label_head_loss', patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_label_head_loss', factor=0.5, patience=3)
]

# 학습
model.fit(train_gen, validation_data=val_gen, epochs=EPOCHS, callbacks=callbacks)

# 평가
preds = model.predict(test_gen)
y_true = test_gen.classes
y_pred = np.argmax(preds[0], axis=1)

f1_macro = f1_score(y_true, y_pred, average='macro')
f1_weighted = f1_score(y_true, y_pred, average='weighted')
print(f"F1-score (macro): {f1_macro:.4f}")
print(f"F1-score (weighted): {f1_weighted:.4f}")



🚀 [START] Stage1 fine-tuning 시작


Found 180 images belonging to 2 classes.
Found 60 images belonging to 2 classes.
Found 60 images belonging to 2 classes.
Epoch 1/30


c:\Users\jh\.conda\envs\tf-gpu\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


6/6 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.0078 - loss: 8.3311 - val_accuracy: 0.0000e+00 - val_loss: 9.5983 - learning_rate: 1.0000e-05
Epoch 2/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.0103 - loss: 7.2950 - val_accuracy: 0.0000e+00 - val_loss: 8.7947 - learning_rate: 1.0000e-05
Epoch 3/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.0251 - loss: 6.5636 - val_accuracy: 0.0000e+00 - val_loss: 7.9874 - learning_rate: 1.0000e-05
Epoch 4/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.0124 - loss: 6.4724 - val_accuracy: 0.0000e+00 - val_loss: 7.1543 - learning_rate: 1.0000e-05
Epoch 5/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.1107 - loss: 5.0849 - val_accuracy: 0.1000 - val_loss: 6.2798 - learning_rate: 1.0000e-05
Epoch 6/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.1255 - loss: 4.3147 - val_accuracy: 0.1667 - val_loss: 5.3507 - learning_rate: 1.0000e-05
Epoch 7/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.1971 - loss: 3.8672 - val_a

💾 [Stage1] Fine-tuned 모델 저장 완료 → anomaly_model/stage1_finetuned_model.h5

📈 [EVAL] Stage1 모델 평가 시작
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step

📄 [Stage1] Classification Report:
              precision    recall  f1-score   support

    Abnormal       0.83      1.00      0.91        20
      Normal       1.00      0.90      0.95        40

    accuracy                           0.93        60
   macro avg       0.92      0.95      0.93        60
weighted avg       0.94      0.93      0.93        60

🧮 [Stage1] F1-score (macro): 0.9282
🧮 [Stage1] F1-score (weighted): 0.9346
📊 Confusion Matrix 저장됨 → reports\Stage1_Confusion_Matrix.png
✅ [Stage1] 완료 및 시각화 저장됨.

🚀 [START] Stage2 fine-tuning 시작


Found 450 images belonging to 3 classes.
Found 150 images belonging to 3 classes.
Found 150 images belonging to 3 classes.
Epoch 1/30


c:\Users\jh\.conda\envs\tf-gpu\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


15/15 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.1921 - loss: 15.9283 - val_accuracy: 0.2933 - val_loss: 12.7973 - learning_rate: 1.0000e-05
Epoch 2/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.2500 - loss: 15.4123 - val_accuracy: 0.3467 - val_loss: 10.5100 - learning_rate: 1.0000e-05
Epoch 3/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.2844 - loss: 12.2516 - val_accuracy: 0.4600 - val_loss: 8.2803 - learning_rate: 1.0000e-05
Epoch 4/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.3251 - loss: 10.2776 - val_accuracy: 0.5667 - val_loss: 5.7731 - learning_rate: 1.0000e-05
Epoch 5/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.3590 - loss: 8.0897 - val_accuracy: 0.6200 - val_loss: 3.6800 - learning_rate: 1.0000e-05
Epoch 6/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.4673 - loss: 5.2568 - val_accuracy: 0.6733 - val_loss: 1.8413 - learning_rate: 1.0000e-05
Epoch 7/30
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 927ms/step - accuracy: 0.5613 - loss: 3.

💾 [Stage2] Fine-tuned 모델 저장 완료 → anomaly_model/stage2_finetuned_model.h5

📈 [EVAL] Stage2 모델 평가 시작
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 609ms/step

📄 [Stage2] Classification Report:
                  precision    recall  f1-score   support

Inner_Race_Fault       0.77      0.98      0.86        50
  Normal_Healthy       1.00      0.50      0.67        50
Outer_Race_Fault       0.80      0.98      0.88        50

        accuracy                           0.82       150
       macro avg       0.86      0.82      0.80       150
    weighted avg       0.86      0.82      0.80       150

🧮 [Stage2] F1-score (macro): 0.8031
🧮 [Stage2] F1-score (weighted): 0.8031
📊 Confusion Matrix 저장됨 → reports\Stage2_Confusion_Matrix.png
✅ [Stage2] 완료 및 시각화 저장됨.


🔍 전체 요약
✅ Stage 1 (Normal vs Abnormal)
Train/Val/Test split: 180 / 60 / 60 (총 300장)

성능 요약:

최종 val_accuracy = 0.8333 → 기준치 0.79 초과 → 조기 종료

Test Accuracy = 0.93

F1 (macro) = 0.9282

F1 (weighted) = 0.9346

Confusion Matrix:

Normal을 Abnormal로 잘못 분류한 비율 거의 없음 (recall 1.00)

성능 매우 우수, 이상 탐지 성공

✅ Stage 2 (Normal vs IR vs OR)
Train/Val/Test split: 450 / 150 / 150 (총 750장)

성능 요약:

최종 val_accuracy = 0.8267 → 기준치 0.79 초과 → 조기 종료

Test Accuracy = 0.82

F1 (macro) = 0.8031

F1 (weighted) = 0.8031

Confusion Matrix 특징:

Normal_Healthy: precision은 1.00이나 recall이 0.50 → 절반만 제대로 잡음

IR, OR: 모두 recall이 0.98로 매우 높음 → fault를 잘 잡아냄

의미:

이상(결함)을 매우 잘 잡지만, 정상을 결함으로 오분류하는 경향 존재 (과탐)

📌 실무 해석 포인트
항목	해석
Stage1	정상/이상 이진 분류 정확도 93%로 매우 높음. 실전 이상탐지에서 사용 가능
Stage2	다중 분류 정확도 82%지만, Normal recall이 낮아 실제 정상 상태를 오탐할 위험 존재
✅ 향후 보완	- Normal 데이터 수 늘리기
- decision threshold 조정
- F1-score 기반 threshold 탐색 필요
- 혼동 행렬 기반 후처리 전략 도입 가능

필요하면 Stage1/2 confusion matrix 시각화 분석, 오탐/미탐 보완 전략까지 이어서 분석해줄 수 있음.
지금 상태는 전체 파인튜닝 루프 완벽히 동작 중이며 성능도 나쁘지 않음.

# val curve 시각화 (acc/loss)

# 예측결과 CSV 저장

# threshold 튜닝 포함